In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
BATCH_SIZE = 10

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

for x, y in test_dataloader:
    print(f"Shape of x [N, C, H, W]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of x [N, C, H, W]: torch.Size([10, 1, 28, 28])
Shape of y: torch.Size([10]) torch.int64


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
from network import Network

model = Network().to(device)
print(model)

Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (simple_sigmoid_stack): Sequential(
    (0): Linear(in_features=784, out_features=30, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=30, out_features=10, bias=True)
    (3): Sigmoid()
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [7]:
def train(dataLoader, model, loss_fn, optimizer):
    size = len(dataLoader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataLoader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------
loss: 2.293765 [   10/60000]
loss: 2.168006 [ 1010/60000]
loss: 1.970805 [ 2010/60000]
loss: 1.932641 [ 3010/60000]
loss: 1.786923 [ 4010/60000]
loss: 1.691782 [ 5010/60000]
loss: 1.705982 [ 6010/60000]
loss: 1.749359 [ 7010/60000]
loss: 1.663068 [ 8010/60000]
loss: 1.718761 [ 9010/60000]
loss: 1.585924 [10010/60000]
loss: 1.577551 [11010/60000]
loss: 1.589456 [12010/60000]
loss: 1.641414 [13010/60000]
loss: 1.700085 [14010/60000]
loss: 1.542679 [15010/60000]
loss: 1.629071 [16010/60000]
loss: 1.588480 [17010/60000]
loss: 1.767465 [18010/60000]
loss: 1.594163 [19010/60000]
loss: 1.597204 [20010/60000]
loss: 1.626982 [21010/60000]
loss: 1.546080 [22010/60000]
loss: 1.614781 [23010/60000]
loss: 1.562791 [24010/60000]
loss: 1.566521 [25010/60000]
loss: 1.512487 [26010/60000]
loss: 1.557719 [27010/60000]
loss: 1.492411 [28010/60000]
loss: 1.677707 [29010/60000]
loss: 1.499198 [30010/60000]
loss: 1.581632 [31010/60000]
loss: 1.497993 [32010/60000]
loss: 1.647278

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted = int(pred[0].argmax(0) )
    print(f"Predicted: {predicted}, Actual: {y}")

Predicted: 7, Actual: 7
